### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [1]:
# #------------------------------------------------------------
# ---------Dependencies and Setup
# #------------------------------------------------------------
import pandas as pd
import numpy as np
from functools import reduce

#variables
school_count = 0
student_count = 0
budget_total = 0
math_score_mean = 0
reading_score_mean = 0
math_pass_perc = 0
reading_pass_perc = 0
both_pass_perc = 0

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_df = pd.read_csv(school_data_to_load)
student_df = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
df = pd.merge(student_df, school_df, how="outer", on="school_name")

# FOR REFERENCE create CSV file of merged data
df.to_csv(path_or_buf = "Resources/merged.csv")


#---------------------------------------------------------
# #------Function to format percentage & thousands sepertor
#---------------------------------------------------------
def perc_formatter(x):
    perc_formatter = "{0:.2%}"
    perc_output = perc_formatter.format(x)
    return(perc_output)

def seperator_formatter(x):
    seperator_formatter = "{:,}"
    seperator_avg_output = seperator_formatter.format(x)
    return(seperator_avg_output)



In [2]:
# #---------------------------------------------------------
# #REF for header columns
# df.columns

# #REF create df for select columns using .loc
# df_grades = df.loc[:, ['Student ID', 'reading_score', 'math_score']]
# df_grades.head()
# #---------------------------------------------------------


#---------------------------------------------------------
# -------SCHOOL OVERALL Summary Calculation-----------------------------
#---------------------------------------------------------
school_count = df["school_name"].nunique()

student_count = df["Student ID"].count()

budget_total = df["budget"].sum()

math_score_mean = df['math_score'].mean() / 100

reading_score_mean = df['reading_score'].mean() / 100

# using Loc on the logic test above only returns rows where the result is True
math_pass = df.loc[df['math_score'] > 69, :]
reading_pass = df.loc[df['reading_score'] > 69, :]

# Multiple conditions can be set to narrow down or widen the filter
both_pass = df.loc[(df['math_score'] > 69) & (df['reading_score'] > 69), :]

# Calculate pass percentages
math_pass_perc = len(math_pass) / student_count
reading_pass_perc = len(reading_pass) / student_count
both_pass_perc = len(both_pass) / student_count


#Format data for results data table.  NOTE:Formatting changes data type
student_count = seperator_formatter(student_count)
budget_total = seperator_formatter(budget_total)
math_score_mean = perc_formatter(math_score_mean)
reading_score_mean = perc_formatter(reading_score_mean)
math_pass_perc = perc_formatter(math_pass_perc)
reading_pass_perc = perc_formatter(reading_pass_perc)
both_pass_perc = perc_formatter(both_pass_perc)

#Create results data frame
results_df = pd.DataFrame({"Total Schools" : [school_count],
                           "Total Students" : [student_count],
                           "Total Budget" : [budget_total],
                           "Average Math Score" : [math_score_mean],
                           "Average Reading Score" : [reading_score_mean],
                           "% Passing Math" : [math_pass_perc],
                           "% Passing Reading" : [reading_pass_perc],
                           "% Overall Passing" : [both_pass_perc]  
})



#display results data frame
results_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","82,932,329,558",78.99%,81.88%,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name x
  * School Type x
  * Total Students x
  * Total School Budget x
  * Per Student Budget 
  * Average Math Score x
  * Average Reading Score x
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
# -----------------------------------------------------
# -----SCHOOL  SUMMARY - START--------------
# -----------------------------------------------------


# #------Using GroupBy in calculate data by "school_name"
# -----------------------------------------------------
g_schoolname_df = df.groupby('school_name')['type'].unique()
# put groupby in dataframe  # also note for Groupby, In order to be visualized, a data function must be used...

school_student_count = df.groupby('school_name')['Student ID'].count()
# school_student_count_df = pd.DataFrame(school_student_count)

school_budget_total = df.groupby('school_name')['budget'].sum()

school_math_avg = df.groupby('school_name')['math_score'].mean()

school_reading_avg = df.groupby('school_name')['reading_score'].mean()

# -----------------------------------------------------
# #------use pd.cut to bin data for PASS FAIL evaluation and add to to df
# -----------------------------------------------------
bins = [0, 69.9, 100]
group_names = [0, 1]

df['Reading Score Summary'] = pd.cut(df["reading_score"].tolist(), bins=bins, labels=group_names)
df['Math Score Summary'] = pd.cut(df["math_score"].tolist(), bins=bins, labels=group_names)

#---------------------------------------
# us "np.where?
# Return elements chosen from `x` or `y` depending on `condition`.
#---------------------------------------
df['Comprehension Score Summary'] = np.where((df['Reading Score Summary'] == 1) & (df['Math Score Summary'] == 1), 1, 0)
df = df.astype({'Reading Score Summary': int, 'Math Score Summary': int, 'Comprehension Score Summary': int})

school_reading_pass = df.groupby('school_name')['Reading Score Summary'].sum()
school_reading_pass = school_reading_pass / school_student_count
school_reading_pass = school_reading_pass.rename('Reading Score Summary')

school_math_pass = df.groupby('school_name')['Math Score Summary'].sum()
school_math_pass = school_math_pass / school_student_count
school_math_pass = school_math_pass.rename('Match Score Summary')

school_comp_pass = df.groupby('school_name')['Comprehension Score Summary'].sum()
school_comp_pass = school_comp_pass / school_student_count
school_comp_pass = school_comp_pass.rename('Comprehension Score Summary')


school_budget_student = school_budget_total / school_student_count
school_budget_student = school_budget_student.rename('Per Student Budget')



# #--------------------------------------------------------------------
# #------MERG dataframe to create School Summary
# ##--------------------------------------------------------------------
g_schoolname_df = pd.merge(g_schoolname_df, school_student_count, how="left", on="school_name")


g_schoolname_df = pd.merge(g_schoolname_df, school_budget_total, how="left", on="school_name")
g_schoolname_df.columns

g_schoolname_df = pd.merge(g_schoolname_df, school_math_avg, how="left", on="school_name")

g_schoolname_df = pd.merge(g_schoolname_df, school_reading_avg, how="left", on="school_name")

g_schoolname_df = pd.merge(g_schoolname_df, school_math_pass, how="left", on="school_name")

g_schoolname_df = pd.merge(g_schoolname_df, school_reading_pass , how="left", on="school_name")

g_schoolname_df = pd.merge(g_schoolname_df, school_comp_pass , how="left", on="school_name")

g_schoolname_df = pd.merge(g_schoolname_df, school_budget_student , how="left", on="school_name")
# #--------------------------------------------------------------------
# #------END MERG dataframe to create School Summary
# ##--------------------------------------------------------------------



# g_schoolname_df.columns

# #--------------------------------------------------------------------
# #------RENAME columns
g_schoolname_df.columns=['School Type', 'Total Students', 'Total School Budget', 'Average Math Score', 'Average Reading Score',
                         '% Passing Math', '% Passing Reading', '% Overall Passing', 'Per Student Budget']

g_schoolname_df = g_schoolname_df.rename_axis("School Name")

# #------RE-ORDER / ORGANIZE columns
g_schoolname_df_reorg = g_schoolname_df[['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 'Average Math Score', 'Average Reading Score',
                         '% Passing Math', '% Passing Reading', '% Overall Passing']]


# #--------------------------------------------------------------------
# # print school summary
g_schoolname_df_reorg



,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,[District],4976,15549641728,3124928.0,77.048432,81.033963,0.666801,0.819333,0.546423
Cabrera High School,[Charter],1858,2009159448,1081356.0,83.061895,83.975780,0.941335,0.970398,0.913348
Figueroa High School,[District],2949,5557128039,1884411.0,76.711767,81.158020,0.659885,0.807392,0.532045
Ford High School,[District],2739,4831365924,1763916.0,77.102592,80.746258,0.683096,0.792990,0.542899
Griffin High School,[Charter],1468,1346890000,917500.0,83.351499,83.816757,0.933924,0.971390,0.905995
Hernandez High School,[District],4635,14007062700,3022020.0,77.289752,80.934412,0.667530,0.808630,0.535275
Holden High School,[Charter],427,105933149,248087.0,83.803279,83.814988,0.925059,0.962529,0.892272
Huang High School,[District],2917,5573322295,1910635.0,76.629414,81.182722,0.656839,0.813164,0.535139
Johnson High School,[District],4761,14733628650,3094650.0,77.072464,80.966394,0.660576,0.812224,0.535392


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
top_schools = g_schoolname_df_reorg.sort_values('% Overall Passing', ascending=False)
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,[Charter],1858,2009159448,1081356.0,83.061895,83.975780,0.941335,0.970398,0.913348
Thomas High School,[Charter],1635,1705517550,1043130.0,83.418349,83.848930,0.932722,0.973089,0.909480
Griffin High School,[Charter],1468,1346890000,917500.0,83.351499,83.816757,0.933924,0.971390,0.905995
Wilson High School,[Charter],2283,3012587442,1319574.0,83.274201,83.989488,0.938677,0.965396,0.905826
Pena High School,[Charter],962,563595396,585858.0,83.839917,84.044699,0.945946,0.959459,0.905405


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
bottom_schools = g_schoolname_df_reorg.sort_values('% Overall Passing')
bottom_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,[District],3999,10186904637,2547363.0,76.842711,80.744686,0.663666,0.802201,0.529882
Figueroa High School,[District],2949,5557128039,1884411.0,76.711767,81.158020,0.659885,0.807392,0.532045
Huang High School,[District],2917,5573322295,1910635.0,76.629414,81.182722,0.656839,0.813164,0.535139
Hernandez High School,[District],4635,14007062700,3022020.0,77.289752,80.934412,0.667530,0.808630,0.535275
Johnson High School,[District],4761,14733628650,3094650.0,77.072464,80.966394,0.660576,0.812224,0.535392


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
# #------------------------------------------------
# #------Reload File to Load to reset data----------------------------
# #---------------------------------- ---------------------------------- 
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_df = pd.read_csv(school_data_to_load)
student_df = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
orig_df = pd.merge(student_df, school_df, how="outer", on="school_name")



In [7]:
# #--------------------------------------------------------
# #------Filter data by grade --------------------------------------------
# #-------------------------------------------------------------------- 
ninth_df = orig_df.query("grade == '9th'")
tenth_df = orig_df.query("grade == '10th'")
eleventh_df = orig_df.query("grade == '11th'")
twelf_df = orig_df.query("grade == '12th'")

# #--------------------------------------------------------
# #------Calculate average by school in each  grade ---------
# #-------------------------------------------------------------------- 
ninth_score_df = ninth_df.groupby('school_name')['math_score'].sum()
ninth_students_df = ninth_df.groupby('school_name')['Student ID'].count()
ninth_avg = ninth_score_df / ninth_students_df
ninth_avg = ninth_avg.rename('9th')

tenth_score_df = tenth_df.groupby('school_name')['math_score'].sum()
tenth_students_df = tenth_df.groupby('school_name')['Student ID'].count()
tenth_avg = tenth_score_df / tenth_students_df
tenth_avg = tenth_avg.rename('10th')

eleventh_score_df = eleventh_df.groupby('school_name')['math_score'].sum()
eleventh_students_df = eleventh_df.groupby('school_name')['Student ID'].count()
eleventh_avg = eleventh_score_df / eleventh_students_df
eleventh_avg = eleventh_avg.rename('11th')

twelf_score_df = twelf_df.groupby('school_name')['math_score'].sum()
twelf_students_df = twelf_df.groupby('school_name')['Student ID'].count()
twelf_avg = twelf_score_df / twelf_students_df
twelf_avg = twelf_avg.rename('12th')



# #--------------------------------------------------------
# #------Merge data ---------
# #-------------------------------------------------------------------- 
m_df = pd.merge(ninth_avg, tenth_avg, how="left", on="school_name")

m_df = pd.merge(m_df, eleventh_avg, how="left", on="school_name")

m_df = pd.merge(m_df, twelf_avg, how="left", on="school_name")

m_df = m_df.rename_axis('School Name')
m_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
# #--------------------------------------------------------
# #------Calculate average by school in each  grade ---------
# #-------------------------------------------------------------------- 
ninth_score_df = ninth_df.groupby('school_name')['reading_score'].sum()
ninth_students_df = ninth_df.groupby('school_name')['Student ID'].count()
ninth_avg = ninth_score_df / ninth_students_df
ninth_avg = ninth_avg.rename('9th')

tenth_score_df = tenth_df.groupby('school_name')['reading_score'].sum()
tenth_students_df = tenth_df.groupby('school_name')['Student ID'].count()
tenth_avg = tenth_score_df / tenth_students_df
tenth_avg = tenth_avg.rename('10th')

eleventh_score_df = eleventh_df.groupby('school_name')['reading_score'].sum()
eleventh_students_df = eleventh_df.groupby('school_name')['Student ID'].count()
eleventh_avg = eleventh_score_df / eleventh_students_df
eleventh_avg = eleventh_avg.rename('11th')

twelf_score_df = twelf_df.groupby('school_name')['reading_score'].sum()
twelf_students_df = twelf_df.groupby('school_name')['Student ID'].count()
twelf_avg = twelf_score_df / twelf_students_df
twelf_avg = twelf_avg.rename('12th')



# #--------------------------------------------------------
# #------Merge data ---------
# #-------------------------------------------------------------------- 
m_df = pd.merge(ninth_avg, tenth_avg, how="left", on="school_name")

m_df = pd.merge(m_df, eleventh_avg, how="left", on="school_name")

m_df = pd.merge(m_df, twelf_avg, how="left", on="school_name")

m_df = m_df.rename_axis('School Name')
m_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [9]:
# # Determine bins for Per budget per student
# orig_df.budget.describe()

# # Determine bins for Per budget per student
# school_budget_student.sort_values()


# orig_df


In [10]:
# Add Merge to add student budget column
orig_df = pd.merge(orig_df,school_budget_student, how='left', on='school_name')

# Add Merge to add count of Score summaries columns.  Key colum is in the loc when targeting columns to add
orig_df = pd.merge(orig_df,df[['Student ID', 'Reading Score Summary', 'Math Score Summary', 'Comprehension Score Summary']], how='left')

# ----------------------------------------------------- 
# #------use pd.cut to bin data and add to to df        
# ----------------------------------------------------- 
bins = [200000, 1000000, 1700000, 2500000, 3200000]
group_names = ["<$200k-1000k", "$1000k-1700k", "$1700k-2500k", "$2500k-3200k"]

orig_df['Per Student Budget Group'] = pd.cut(orig_df['Per Student Budget'].tolist(), bins=bins, labels=group_names)

In [11]:
#  orig_df.head()

In [12]:
group_m_mean = orig_df.groupby('Per Student Budget Group')['math_score'].mean()
group_r_mean = orig_df.groupby('Per Student Budget Group')['reading_score'].mean()

group_m_pass = orig_df.groupby('Per Student Budget Group')['Reading Score Summary'].sum()
group_r_pass = orig_df.groupby('Per Student Budget Group')['Math Score Summary'].sum()
group_c_pass = orig_df.groupby('Per Student Budget Group')['Comprehension Score Summary'].sum()
group_students = orig_df.groupby('Per Student Budget Group')['Student ID'].count()

group_m_percpass = group_m_pass / group_students
group_m_percpass = group_m_percpass.rename("% Passing Math")

group_r_percpass = group_r_pass / group_students
group_r_percpass = group_r_percpass.rename("% Passing Reading")

group_c_percpass = group_c_pass / group_students
group_c_percpass = group_c_percpass.rename("% Overall Passing")

spendsummary_df = pd.merge(group_m_mean, group_r_mean, how='left', on='Per Student Budget Group')

spendsummary_df = pd.merge(spendsummary_df, group_m_mean, how='left', on='Per Student Budget Group')
spendsummary_df = pd.merge(spendsummary_df, group_m_percpass, how='left', on='Per Student Budget Group')
spendsummary_df = pd.merge(spendsummary_df, group_r_percpass, how='left', on='Per Student Budget Group')
spendsummary_df = pd.merge(spendsummary_df, group_c_percpass, how='left', on='Per Student Budget Group')

In [13]:
spendsummary_df.drop(labels='math_score_y', axis=1, inplace=True)
spendsummary_df.columns = [['Average Math_Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']]
spendsummary_df

,Average Math_Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Per Student Budget Group,,,,,
<$200k-1000k,83.583479,83.893245,0.966048,0.936647,0.903745
$1000k-1700k,83.351933,83.905751,0.966585,0.937132,0.906180
$1700k-2500k,76.808251,81.035328,0.804765,0.666241,0.536549
$2500k-3200k,77.070764,80.928365,0.811061,0.664689,0.537151


In [14]:
# group_r_mean.head()

## Scores by School Size

* Perform the same operations as above, based on school size.

In [15]:
# orig_df['size'].describe()

In [16]:
# ----------------------------------------------------- 
# #------use pd.cut to bin data and add to to df        
# ----------------------------------------------------- 

bins = [400, 1800, 3000, 4000, 5000]
group_names = ['400-1800', '1800-3000', '3000-4000', '4000-5000']

orig_df['School Size Group'] = pd.cut(orig_df['size'].tolist(), bins=bins, labels=group_names)



In [17]:
group_m_mean = orig_df.groupby('School Size Group')['math_score'].mean()
group_r_mean = orig_df.groupby('School Size Group')['reading_score'].mean()



In [18]:
group_m_mean = orig_df.groupby('School Size Group')['math_score'].mean()
group_r_mean = orig_df.groupby('School Size Group')['reading_score'].mean()

group_m_pass = orig_df.groupby('School Size Group')['Reading Score Summary'].sum()
group_r_pass = orig_df.groupby('School Size Group')['Math Score Summary'].sum()
group_c_pass = orig_df.groupby('School Size Group')['Comprehension Score Summary'].sum()
group_students = orig_df.groupby('School Size Group')['Student ID'].count()

group_m_percpass = group_m_pass / group_students
group_m_percpass = group_m_percpass.rename("% Passing Math")

group_r_percpass = group_r_pass / group_students
group_r_percpass = group_r_percpass.rename("% Passing Reading")

group_c_percpass = group_c_pass / group_students
group_c_percpass = group_c_percpass.rename("% Overall Passing")

spendsummary_df_2 = pd.merge(group_m_mean, group_r_mean, how='left', on='School Size Group')
spendsummary_df_2 = pd.merge(spendsummary_df_2, group_m_mean, how='left', on='School Size Group')
spendsummary_df_2 = pd.merge(spendsummary_df_2, group_m_percpass, how='left', on='School Size Group')
spendsummary_df_2 = pd.merge(spendsummary_df_2, group_r_percpass, how='left', on='School Size Group')
spendsummary_df_2 = pd.merge(spendsummary_df_2, group_c_percpass, how='left', on='School Size Group')

In [19]:
spendsummary_df_2.drop(labels='math_score_y', axis=1, inplace=True)
spendsummary_df_2.columns = [['Average Math_Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']]
spendsummary_df_2

,Average Math_Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size Group,,,,,
400-1800,83.523035,83.861418,0.965851,0.935552,0.903763
1800-3000,78.878001,81.993096,0.857681,0.755139,0.657618
3000-4000,76.842711,80.744686,0.802201,0.663666,0.529882
4000-5000,77.134219,80.979474,0.813526,0.664974,0.539173


In [20]:
# ------------------------------------------------------------------------------
# ------------------------------------------------------------------------------
# #-----------------?????????????QUESTION CAN I CREATE A FUNCION ON THIS---------------------------------------
# ------------------------------------------------------------------------------
# ------------------------------------------------------------------------------
# def scores_summary_by(simple):
#     group_m_mean = orig_df.groupby(simple)['math_score'].mean()
#     group_r_mean = orig_df.groupby(simple)['reading_score'].mean()

#     group_m_pass = orig_df.groupby(simple)['Reading Score Summary'].sum()
#     group_r_pass = orig_df.groupby(simple)['Math Score Summary'].sum()
#     group_c_pass = orig_df.groupby(simple)['Comprehension Score Summary'].sum()
#     group_students = orig_df.groupby(simple)['Student ID'].count()
    
#     group_m_percpass = group_m_pass / group_students
#     group_m_percpass = group_m_percpass.rename("% Passing Math")

#     group_r_percpass = group_r_pass / group_students
#     group_r_percpass = group_r_percpass.rename("% Passing Reading")

#     group_c_percpass = group_c_pass / group_students
#     group_c_percpass = group_c_percpass.rename("% Overall Passing")

#     spendsummary_df = pd.merge(group_m_mean, group_r_mean, how='left', on=simple)
#     spendsummary_df = pd.merge(spendsummary_df, group_m_mean, how='left', on=simple)
#     spendsummary_df = pd.merge(spendsummary_df, group_m_percpass, how='left', on=simple)
#     spendsummary_df = pd.merge(spendsummary_df, group_r_percpass, how='left', on=simple)
#     spendsummary_df = pd.merge(spendsummary_df, group_c_percpass, how='left', on=simple)
    
#     return(spendsummary_df_v)

# a = "School Size Group"

# scores_summary_by(a)
# #-----------------?????????????QUESTION CAN I CREATE A FUNCION ON THIS---------------------------------------
# ------------------------------------------------------------------------------
# ------------------------------------------------------------------------------

## Scores by School Type

* Perform the same operations as above, based on school type

In [21]:
# orig_df.head()

In [22]:
# orig_df['type'].describe

In [23]:
type_math_avg = orig_df.groupby('type')['math_score'].mean()
type_reading_avg = orig_df.groupby('type')['reading_score'].mean()

In [24]:
type_math_pass = orig_df.groupby('type')['Math Score Summary'].sum()
type_reading_pass = orig_df.groupby('type')['Reading Score Summary'].sum()
type_comp_pass = orig_df.groupby('type')['Comprehension Score Summary'].sum()
group_students = orig_df.groupby('type')['Student ID'].count()

type_math_passperc = type_math_pass / group_students

In [25]:
type_math_passperc = type_math_passperc.rename("% Passing Math")

In [26]:
type_reading_passperc = type_reading_pass / group_students
type_reading_passperc = type_reading_passperc.rename("% Passing Reading")
type_comp_passperc = type_comp_pass / group_students
type_comp_passperc =  type_comp_passperc.rename("% Overall Passing")

In [27]:
type_summary_df = pd.merge(type_math_avg, type_reading_avg, how="left", on="type")

In [28]:
type_summary_df = pd.merge(type_summary_df, type_math_passperc, how="left", on="type")

In [29]:
type_summary_df = pd.merge(type_summary_df, type_reading_passperc, how="left", on="type")
type_summary_df = pd.merge(type_summary_df, type_comp_passperc, how="left", on="type")

In [30]:
# #------rename columns
type_summary_df.columns=[['Average Math_Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']]


In [31]:
type_summary_df 

,Average Math_Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.406183,83.902821,0.937018,0.966459,0.905609
District,76.987026,80.962485,0.665184,0.809052,0.536959
